# Mumbai Colleges Placement Data Analysis
## Comprehensive Analysis & Visualization

This notebook provides a complete analysis of placement data from Mumbai colleges, examining:
- Placement rates across colleges, fields, and years
- Average package trends and comparisons
- Statistical insights and correlations
- Interactive visualizations for data exploration

## 1. Import Required Libraries

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configure visualization settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# For interactive visualizations
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Plotly not available. Install with: pip install plotly")

print("✓ Libraries imported successfully!")

## 2. Load and Explore the Dataset

In [ ]:
# Load the placement data
df = pd.read_csv('placements.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\nColumn Names and Data Types:")
print(df.dtypes)
print("\n" + "="*80)
print("First 10 rows:")
display(df.head(10))

print("\n" + "="*80)
print("Last 5 rows:")
display(df.tail())

print("\n" + "="*80)
print("Dataset Info:")
df.info()

## 3. Data Cleaning and Feature Engineering

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates}")

# Create derived features
df['Placement_Rate'] = (df['Placed_Students'] / df['Total_Students']) * 100
df['Unplaced_Students'] = df['Total_Students'] - df['Placed_Students']

# Create tier classification based on average package
def classify_tier(avg_package):
    if avg_package >= 12:
        return 'Tier 1 (Premium)'
    elif avg_package >= 7:
        return 'Tier 2 (High)'
    else:
        return 'Tier 3 (Average)'

df['Tier'] = df['Avg_Package_LPA'].apply(classify_tier)

# Display the enhanced dataset
print("\n" + "="*80)
print("Enhanced Dataset with Derived Features:")
display(df.head(10))

print("\n" + "="*80)
print("Summary of New Features:")
print(f"Placement Rate - Min: {df['Placement_Rate'].min():.2f}%, Max: {df['Placement_Rate'].max():.2f}%")
print(f"\nTier Distribution:")
print(df['Tier'].value_counts().sort_index())

## 4. Statistical Summary

In [ ]:
# Descriptive statistics
print("Descriptive Statistics:")
display(df.describe())

print("\n" + "="*80)
print("Key Metrics Summary:")
print(f"Total Records: {len(df)}")
print(f"Unique Colleges: {df['College'].nunique()}")
print(f"Unique Fields: {df['Field'].nunique()}")
print(f"Years Covered: {sorted(df['Year'].unique())}")

print("\n" + "="*80)
print("Overall Placement Metrics:")
print(f"Total Students: {df['Total_Students'].sum():,}")
print(f"Total Placed Students: {df['Placed_Students'].sum():,}")
print(f"Overall Placement Rate: {(df['Placed_Students'].sum() / df['Total_Students'].sum() * 100):.2f}%")
print(f"Average Package (Mean): ₹{df['Avg_Package_LPA'].mean():.2f} LPA")
print(f"Average Package (Median): ₹{df['Avg_Package_LPA'].median():.2f} LPA")
print(f"Highest Package Offered: ₹{df['Avg_Package_LPA'].max():.2f} LPA")
print(f"Lowest Package Offered: ₹{df['Avg_Package_LPA'].min():.2f} LPA")

## 5. College-wise Placement Analysis

In [ ]:
# College-wise aggregation
college_stats = df.groupby('College').agg({
    'Total_Students': 'sum',
    'Placed_Students': 'sum',
    'Avg_Package_LPA': 'mean',
    'Placement_Rate': 'mean'
}).round(2)

college_stats = college_stats.sort_values('Placement_Rate', ascending=False)

print("College-wise Placement Statistics (sorted by Placement Rate):")
display(college_stats)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Placement Rate by College
college_stats.sort_values('Placement_Rate', ascending=True).plot(
    y='Placement_Rate', kind='barh', ax=axes[0, 0], color='skyblue', legend=False
)
axes[0, 0].set_title('Placement Rate by College', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Placement Rate (%)')
axes[0, 0].grid(axis='x', alpha=0.3)

# 2. Average Package by College
college_stats.sort_values('Avg_Package_LPA', ascending=True).plot(
    y='Avg_Package_LPA', kind='barh', ax=axes[0, 1], color='lightcoral', legend=False
)
axes[0, 1].set_title('Average Package by College', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Average Package (LPA)')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Total vs Placed Students
college_stats_sorted = college_stats.sort_values('Total_Students', ascending=False).head(10)
x = np.arange(len(college_stats_sorted))
width = 0.35
axes[1, 0].bar(x - width/2, college_stats_sorted['Total_Students'], width, label='Total Students', color='steelblue')
axes[1, 0].bar(x + width/2, college_stats_sorted['Placed_Students'], width, label='Placed Students', color='green')
axes[1, 0].set_title('Top 10 Colleges: Total vs Placed Students', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(college_stats_sorted.index, rotation=45, ha='right')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. College Performance Matrix
college_perf = df.groupby('College')['Placement_Rate'].mean().sort_values(ascending=False)
axes[1, 1].scatter(college_stats['Avg_Package_LPA'], college_stats['Placement_Rate'],
                   s=college_stats['Total_Students']/5, alpha=0.6, c=range(len(college_stats)), cmap='viridis')
axes[1, 1].set_title('College Performance Matrix\n(Size = Total Students)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Average Package (LPA)')
axes[1, 1].set_ylabel('Placement Rate (%)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Field-wise Placement Analysis

In [ ]:
# Field-wise aggregation
field_stats = df.groupby('Field').agg({
    'Total_Students': 'sum',
    'Placed_Students': 'sum',
    'Avg_Package_LPA': 'mean',
    'Placement_Rate': 'mean'
}).round(2)

field_stats = field_stats.sort_values('Avg_Package_LPA', ascending=False)

print("Field-wise Placement Statistics (sorted by Average Package):")
display(field_stats.head(15))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Top Fields by Average Package
top_fields_pkg = field_stats.nlargest(10, 'Avg_Package_LPA')
top_fields_pkg.plot(y='Avg_Package_LPA', kind='barh', ax=axes[0, 0], color='gold', legend=False)
axes[0, 0].set_title('Top 10 Fields by Average Package', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Average Package (LPA)')
axes[0, 0].grid(axis='x', alpha=0.3)

# 2. Top Fields by Placement Rate
top_fields_rate = field_stats.nlargest(10, 'Placement_Rate')
top_fields_rate.plot(y='Placement_Rate', kind='barh', ax=axes[0, 1], color='lightgreen', legend=False)
axes[0, 1].set_title('Top 10 Fields by Placement Rate', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Placement Rate (%)')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Field Distribution - Pie Chart
field_student_count = df.groupby('Field')['Total_Students'].sum().nlargest(8)
axes[1, 0].pie(field_student_count.values, labels=field_student_count.index, autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Top 8 Fields by Student Count', fontsize=14, fontweight='bold')

# 4. Package vs Placement Rate by Field
axes[1, 1].scatter(field_stats['Avg_Package_LPA'], field_stats['Placement_Rate'],
                   s=field_stats['Total_Students']/3, alpha=0.6, c=range(len(field_stats)), cmap='plasma')
axes[1, 1].set_title('Field Performance: Package vs Placement Rate', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Average Package (LPA)')
axes[1, 1].set_ylabel('Placement Rate (%)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Year-wise Trend Analysis

In [ ]:
# Year-wise analysis
year_stats = df.groupby('Year').agg({
    'Total_Students': 'sum',
    'Placed_Students': 'sum',
    'Avg_Package_LPA': 'mean',
    'Placement_Rate': 'mean'
}).round(2)

print("Year-wise Placement Trends:")
display(year_stats)

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Placement Rate Trend
axes[0, 0].plot(year_stats.index, year_stats['Placement_Rate'], marker='o', linewidth=2, markersize=8, color='blue')
axes[0, 0].set_title('Placement Rate Trend (2023-2025)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Placement Rate (%)')
axes[0, 0].grid(alpha=0.3)
for i, v in enumerate(year_stats['Placement_Rate']):
    axes[0, 0].text(year_stats.index[i], v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

# 2. Average Package Trend
axes[0, 1].plot(year_stats.index, year_stats['Avg_Package_LPA'], marker='s', linewidth=2, markersize=8, color='green')
axes[0, 1].set_title('Average Package Trend (2023-2025)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Package (LPA)')
axes[0, 1].grid(alpha=0.3)
for i, v in enumerate(year_stats['Avg_Package_LPA']):
    axes[0, 1].text(year_stats.index[i], v + 0.2, f'₹{v:.2f}', ha='center', fontweight='bold')

# 3. Student Count Trend
width = 0.35
x = np.arange(len(year_stats))
axes[1, 0].bar(x - width/2, year_stats['Total_Students'], width, label='Total Students', color='steelblue')
axes[1, 0].bar(x + width/2, year_stats['Placed_Students'], width, label='Placed Students', color='orange')
axes[1, 0].set_title('Total vs Placed Students by Year', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Number of Students')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(year_stats.index)
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Year-wise College Count
college_year_count = df.groupby('Year')['College'].nunique()
axes[1, 1].bar(college_year_count.index, college_year_count.values, color='purple', alpha=0.7)
axes[1, 1].set_title('Number of Colleges Reporting by Year', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Number of Colleges')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Package Distribution Analysis

In [ ]:
# Package distribution visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Histogram of Average Packages
axes[0, 0].hist(df['Avg_Package_LPA'], bins=30, color='teal', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Distribution of Average Packages', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Average Package (LPA)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['Avg_Package_LPA'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ₹{df["Avg_Package_LPA"].mean():.2f}')
axes[0, 0].axvline(df['Avg_Package_LPA'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: ₹{df["Avg_Package_LPA"].median():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Box Plot by Tier
tier_order = ['Tier 1 (Premium)', 'Tier 2 (High)', 'Tier 3 (Average)']
sns.boxplot(data=df, y='Tier', x='Avg_Package_LPA', order=tier_order, palette='coolwarm', ax=axes[0, 1])
axes[0, 1].set_title('Package Distribution by Tier', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Average Package (LPA)')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Violin Plot by Year
sns.violinplot(data=df, x='Year', y='Avg_Package_LPA', palette='Set2', ax=axes[1, 0])
axes[1, 0].set_title('Package Distribution by Year', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Average Package (LPA)')
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Top 10 Highest Average Packages
top_packages = df.nlargest(10, 'Avg_Package_LPA')[['College', 'Field', 'Year', 'Avg_Package_LPA']]
top_packages['Label'] = top_packages['College'].str[:15] + '\n' + top_packages['Field'].str[:20]
axes[1, 1].barh(range(10), top_packages['Avg_Package_LPA'].values, color='crimson', alpha=0.7)
axes[1, 1].set_yticks(range(10))
axes[1, 1].set_yticklabels(top_packages['Label'].values, fontsize=8)
axes[1, 1].set_title('Top 10 Highest Average Packages', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Average Package (LPA)')
axes[1, 1].invert_yaxis()
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPackage Statistics by Tier:")
print(df.groupby('Tier')['Avg_Package_LPA'].describe())

## 9. Top Performers Analysis

In [ ]:
# Top performers by different metrics
print("="*80)
print("TOP 10 COLLEGES BY PLACEMENT RATE:")
print("="*80)
top_placement_colleges = df.groupby('College').agg({
    'Placement_Rate': 'mean',
    'Avg_Package_LPA': 'mean',
    'Total_Students': 'sum'
}).round(2).sort_values('Placement_Rate', ascending=False).head(10)
display(top_placement_colleges)

print("\n" + "="*80)
print("TOP 10 COLLEGES BY AVERAGE PACKAGE:")
print("="*80)
top_package_colleges = df.groupby('College').agg({
    'Avg_Package_LPA': 'mean',
    'Placement_Rate': 'mean',
    'Total_Students': 'sum'
}).round(2).sort_values('Avg_Package_LPA', ascending=False).head(10)
display(top_package_colleges)

print("\n" + "="*80)
print("TOP 10 FIELDS BY PLACEMENT RATE:")
print("="*80)
top_placement_fields = df.groupby('Field').agg({
    'Placement_Rate': 'mean',
    'Avg_Package_LPA': 'mean',
    'Total_Students': 'sum'
}).round(2).sort_values('Placement_Rate', ascending=False).head(10)
display(top_placement_fields)

print("\n" + "="*80)
print("TOP 10 FIELDS BY AVERAGE PACKAGE:")
print("="*80)
top_package_fields = df.groupby('Field').agg({
    'Avg_Package_LPA': 'mean',
    'Placement_Rate': 'mean',
    'Total_Students': 'sum'
}).round(2).sort_values('Avg_Package_LPA', ascending=False).head(10)
display(top_package_fields)

# Visualization of top performers
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Top 10 Colleges by Placement Rate
top_placement_colleges.plot(y='Placement_Rate', kind='bar', ax=axes[0, 0], color='seagreen', legend=False)
axes[0, 0].set_title('Top 10 Colleges by Placement Rate', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Placement Rate (%)')
axes[0, 0].set_xlabel('')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Top 10 Colleges by Package
top_package_colleges.plot(y='Avg_Package_LPA', kind='bar', ax=axes[0, 1], color='coral', legend=False)
axes[0, 1].set_title('Top 10 Colleges by Average Package', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Average Package (LPA)')
axes[0, 1].set_xlabel('')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Top 10 Fields by Placement Rate
top_placement_fields.plot(y='Placement_Rate', kind='barh', ax=axes[1, 0], color='dodgerblue', legend=False)
axes[1, 0].set_title('Top 10 Fields by Placement Rate', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Placement Rate (%)')
axes[1, 0].grid(axis='x', alpha=0.3)

# 4. Top 10 Fields by Package
top_package_fields.plot(y='Avg_Package_LPA', kind='barh', ax=axes[1, 1], color='gold', legend=False)
axes[1, 1].set_title('Top 10 Fields by Average Package', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Average Package (LPA)')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Correlation and Relationship Analysis

In [ ]:
# Correlation analysis
numeric_cols = ['Total_Students', 'Placed_Students', 'Avg_Package_LPA', 'Placement_Rate']
correlation_matrix = df[numeric_cols].corr()

print("Correlation Matrix:")
display(correlation_matrix)

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Correlation Heatmap
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, ax=axes[0, 0], cbar_kws={"shrink": 0.8})
axes[0, 0].set_title('Correlation Matrix of Key Metrics', fontsize=14, fontweight='bold')

# 2. Placement Rate vs Average Package
axes[0, 1].scatter(df['Avg_Package_LPA'], df['Placement_Rate'], alpha=0.6, c=df['Year'], cmap='viridis', s=100)
axes[0, 1].set_title('Placement Rate vs Average Package', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Average Package (LPA)')
axes[0, 1].set_ylabel('Placement Rate (%)')
axes[0, 1].grid(alpha=0.3)
cbar = plt.colorbar(axes[0, 1].collections[0], ax=axes[0, 1])
cbar.set_label('Year')

# 3. Total Students vs Placement Rate
axes[1, 0].scatter(df['Total_Students'], df['Placement_Rate'], alpha=0.6, c=df['Avg_Package_LPA'], cmap='plasma', s=100)
axes[1, 0].set_title('Total Students vs Placement Rate', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Total Students')
axes[1, 0].set_ylabel('Placement Rate (%)')
axes[1, 0].grid(alpha=0.3)
cbar = plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])
cbar.set_label('Avg Package (LPA)')

# 4. Total Students vs Average Package
axes[1, 1].scatter(df['Total_Students'], df['Avg_Package_LPA'], alpha=0.6, c=df['Placement_Rate'], cmap='RdYlGn', s=100)
axes[1, 1].set_title('Total Students vs Average Package', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Total Students')
axes[1, 1].set_ylabel('Average Package (LPA)')
axes[1, 1].grid(alpha=0.3)
cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
cbar.set_label('Placement Rate (%)')

plt.tight_layout()
plt.show()

## 11. Comprehensive Heatmap Analysis

In [ ]:
# Create pivot tables for heatmaps
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# 1. College vs Field Heatmap - Placement Rate
# Get top colleges and fields for better visualization
top_colleges = df.groupby('College')['Placed_Students'].sum().nlargest(8).index
top_fields = df.groupby('Field')['Total_Students'].sum().nlargest(8).index

heatmap_data_rate = df[df['College'].isin(top_colleges) & df['Field'].isin(top_fields)].pivot_table(
    values='Placement_Rate',
    index='College',
    columns='Field',
    aggfunc='mean'
)

sns.heatmap(heatmap_data_rate, annot=True, fmt='.1f', cmap='YlGnBu', ax=axes[0],
            cbar_kws={'label': 'Placement Rate (%)'}, linewidths=0.5)
axes[0].set_title('Placement Rate Heatmap\n(Top Colleges vs Top Fields)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Field')
axes[0].set_ylabel('College')

# 2. College vs Field Heatmap - Average Package
heatmap_data_package = df[df['College'].isin(top_colleges) & df['Field'].isin(top_fields)].pivot_table(
    values='Avg_Package_LPA',
    index='College',
    columns='Field',
    aggfunc='mean'
)

sns.heatmap(heatmap_data_package, annot=True, fmt='.2f', cmap='RdYlGn', ax=axes[1],
            cbar_kws={'label': 'Avg Package (LPA)'}, linewidths=0.5)
axes[1].set_title('Average Package Heatmap\n(Top Colleges vs Top Fields)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Field')
axes[1].set_ylabel('College')

plt.tight_layout()
plt.show()

# Additional heatmap: Year vs College
fig, ax = plt.subplots(figsize=(14, 8))
year_college_pivot = df.groupby(['Year', 'College'])['Placement_Rate'].mean().unstack(fill_value=0)
sns.heatmap(year_college_pivot.T, annot=True, fmt='.1f', cmap='viridis', ax=ax,
            cbar_kws={'label': 'Placement Rate (%)'}, linewidths=0.5)
ax.set_title('Year-wise Placement Rate by College', fontsize=14, fontweight='bold')
ax.set_xlabel('Year')
ax.set_ylabel('College')
plt.tight_layout()
plt.show()

## 12. Comparative Analysis: Computer Science vs Other Fields

In [ ]:
# Define key engineering fields for comparison
key_fields = ['Computer Science', 'Information Technology', 'Mechanical', 'Civil',
              'Electrical', 'Electronics', 'Electronics & Telecom']

# Filter data for key fields
df_key_fields = df[df['Field'].isin(key_fields)]

# Aggregate statistics by field
field_comparison = df_key_fields.groupby('Field').agg({
    'Total_Students': 'sum',
    'Placed_Students': 'sum',
    'Avg_Package_LPA': 'mean',
    'Placement_Rate': 'mean'
}).round(2).sort_values('Avg_Package_LPA', ascending=False)

print("Comparison of Key Engineering Fields:")
display(field_comparison)

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Average Package Comparison
field_comparison.sort_values('Avg_Package_LPA').plot(
    y='Avg_Package_LPA', kind='barh', ax=axes[0, 0], color='steelblue', legend=False
)
axes[0, 0].set_title('Average Package by Key Fields', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Average Package (LPA)')
axes[0, 0].grid(axis='x', alpha=0.3)

# 2. Placement Rate Comparison
field_comparison.sort_values('Placement_Rate').plot(
    y='Placement_Rate', kind='barh', ax=axes[0, 1], color='orange', legend=False
)
axes[0, 1].set_title('Placement Rate by Key Fields', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Placement Rate (%)')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Student Distribution
field_comparison.sort_values('Total_Students', ascending=False).plot(
    y='Total_Students', kind='bar', ax=axes[1, 0], color='green', legend=False
)
axes[1, 0].set_title('Total Students by Field', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Total Students')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Field Performance Matrix
axes[1, 1].scatter(field_comparison['Avg_Package_LPA'], field_comparison['Placement_Rate'],
                   s=field_comparison['Total_Students']/3, alpha=0.6, c=range(len(field_comparison)), cmap='rainbow')
for idx, field in enumerate(field_comparison.index):
    axes[1, 1].annotate(field,
                        (field_comparison.loc[field, 'Avg_Package_LPA'],
                         field_comparison.loc[field, 'Placement_Rate']),
                        fontsize=9, alpha=0.7)
axes[1, 1].set_title('Field Performance Matrix (Size = Total Students)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Average Package (LPA)')
axes[1, 1].set_ylabel('Placement Rate (%)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 13. College Benchmarking Analysis

In [ ]:
# College performance benchmarking
colleges_to_compare = ['VJTI', 'SPIT', 'VESIT', 'VIT Mumbai', 'Fr. Conceicao Rodrigues College of Engineering', 'ACPCE']

# Filter data for selected colleges
df_benchmark = df[df['College'].isin(colleges_to_compare)]

# Multi-metric comparison
benchmark_stats = df_benchmark.groupby('College').agg({
    'Placement_Rate': 'mean',
    'Avg_Package_LPA': 'mean',
    'Total_Students': 'sum',
    'Placed_Students': 'sum'
}).round(2)

print("Benchmarking Selected Colleges:")
display(benchmark_stats.sort_values('Avg_Package_LPA', ascending=False))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Multi-metric Radar/Spider Chart comparison (simplified as grouped bar)
benchmark_normalized = benchmark_stats.copy()
for col in benchmark_normalized.columns:
    benchmark_normalized[col] = (benchmark_normalized[col] - benchmark_normalized[col].min()) / \
                                 (benchmark_normalized[col].max() - benchmark_normalized[col].min()) * 100

x = np.arange(len(benchmark_stats))
width = 0.2
axes[0, 0].bar(x - 1.5*width, benchmark_normalized['Placement_Rate'], width, label='Placement Rate')
axes[0, 0].bar(x - 0.5*width, benchmark_normalized['Avg_Package_LPA'], width, label='Avg Package')
axes[0, 0].bar(x + 0.5*width, benchmark_normalized['Total_Students'], width, label='Total Students (scaled)')
axes[0, 0].bar(x + 1.5*width, benchmark_normalized['Placed_Students'], width, label='Placed Students (scaled)')
axes[0, 0].set_title('Normalized Multi-Metric Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(benchmark_stats.index, rotation=45, ha='right')
axes[0, 0].set_ylabel('Normalized Score (0-100)')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Placement Rate Comparison
benchmark_stats.sort_values('Placement_Rate').plot(
    y='Placement_Rate', kind='barh', ax=axes[0, 1], color='teal', legend=False
)
axes[0, 1].set_title('Placement Rate Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Placement Rate (%)')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Average Package Comparison
benchmark_stats.sort_values('Avg_Package_LPA').plot(
    y='Avg_Package_LPA', kind='barh', ax=axes[1, 0], color='crimson', legend=False
)
axes[1, 0].set_title('Average Package Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Average Package (LPA)')
axes[1, 0].grid(axis='x', alpha=0.3)

# 4. Scatter: Package vs Placement Rate
for college in benchmark_stats.index:
    college_data = benchmark_stats.loc[college]
    axes[1, 1].scatter(college_data['Avg_Package_LPA'], college_data['Placement_Rate'],
                       s=college_data['Total_Students']/2, alpha=0.6, label=college)
axes[1, 1].set_title('Package vs Placement Rate\n(Bubble size = Total Students)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Average Package (LPA)')
axes[1, 1].set_ylabel('Placement Rate (%)')
axes[1, 1].legend(fontsize=8, loc='best')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 14. Advanced Insights & Key Findings

In [ ]:
# Generate comprehensive insights
print("="*80)
print("KEY FINDINGS AND INSIGHTS")
print("="*80)

# 1. Best performing college
best_college_rate = df.groupby('College')['Placement_Rate'].mean().idxmax()
best_college_rate_val = df.groupby('College')['Placement_Rate'].mean().max()
print(f"\n✓ Best Placement Rate: {best_college_rate} ({best_college_rate_val:.2f}%)")

best_college_pkg = df.groupby('College')['Avg_Package_LPA'].mean().idxmax()
best_college_pkg_val = df.groupby('College')['Avg_Package_LPA'].mean().max()
print(f"✓ Highest Average Package: {best_college_pkg} (₹{best_college_pkg_val:.2f} LPA)")

# 2. Best performing field
best_field_rate = df.groupby('Field')['Placement_Rate'].mean().idxmax()
best_field_rate_val = df.groupby('Field')['Placement_Rate'].mean().max()
print(f"\n✓ Field with Best Placement Rate: {best_field_rate} ({best_field_rate_val:.2f}%)")

best_field_pkg = df.groupby('Field')['Avg_Package_LPA'].mean().idxmax()
best_field_pkg_val = df.groupby('Field')['Avg_Package_LPA'].mean().max()
print(f"✓ Field with Highest Package: {best_field_pkg} (₹{best_field_pkg_val:.2f} LPA)")

# 3. Year-over-year trends
print("\n" + "="*80)
print("YEAR-OVER-YEAR TRENDS:")
print("="*80)
year_trends = df.groupby('Year').agg({
    'Placement_Rate': 'mean',
    'Avg_Package_LPA': 'mean'
}).round(2)
for year in sorted(df['Year'].unique()):
    print(f"Year {year}: Placement Rate = {year_trends.loc[year, 'Placement_Rate']:.2f}%, "
          f"Avg Package = ₹{year_trends.loc[year, 'Avg_Package_LPA']:.2f} LPA")

# 4. Tier distribution insights
print("\n" + "="*80)
print("TIER DISTRIBUTION:")
print("="*80)
tier_dist = df['Tier'].value_counts().sort_index()
for tier, count in tier_dist.items():
    percentage = (count / len(df)) * 100
    print(f"{tier}: {count} records ({percentage:.1f}%)")

# 5. Computer Science vs Other Fields
print("\n" + "="*80)
print("COMPUTER SCIENCE COMPARISON:")
print("="*80)
cs_fields = df[df['Field'].str.contains('Computer', case=False, na=False)]
non_cs_fields = df[~df['Field'].str.contains('Computer', case=False, na=False)]

print(f"CS-related fields - Avg Placement Rate: {cs_fields['Placement_Rate'].mean():.2f}%")
print(f"CS-related fields - Avg Package: ₹{cs_fields['Avg_Package_LPA'].mean():.2f} LPA")
print(f"Non-CS fields - Avg Placement Rate: {non_cs_fields['Placement_Rate'].mean():.2f}%")
print(f"Non-CS fields - Avg Package: ₹{non_cs_fields['Avg_Package_LPA'].mean():.2f} LPA")

# 6. Records with 100% placement
perfect_placement = df[df['Placement_Rate'] == 100]
print(f"\n✓ Records with 100% Placement: {len(perfect_placement)}")
if len(perfect_placement) > 0:
    print("\nColleges/Fields with Perfect Placement:")
    display(perfect_placement[['College', 'Field', 'Year', 'Total_Students', 'Avg_Package_LPA']].head(10))

# 7. Underperforming areas (placement rate < 50%)
low_placement = df[df['Placement_Rate'] < 50]
print(f"\n✓ Records with Placement Rate < 50%: {len(low_placement)}")
if len(low_placement) > 0:
    print("\nAreas Needing Improvement:")
    display(low_placement[['College', 'Field', 'Year', 'Placement_Rate', 'Avg_Package_LPA']].head(10))

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)

## 15. Interactive Visualizations (Plotly)
*These visualizations are interactive - hover, zoom, and filter dynamically!*

In [ ]:
if PLOTLY_AVAILABLE:
    # 1. Interactive Scatter Plot: Placement Rate vs Average Package
    fig = px.scatter(df,
                     x='Avg_Package_LPA',
                     y='Placement_Rate',
                     size='Total_Students',
                     color='College',
                     hover_data=['Field', 'Year', 'Placed_Students'],
                     title='Interactive: Placement Rate vs Average Package',
                     labels={'Avg_Package_LPA': 'Average Package (LPA)',
                            'Placement_Rate': 'Placement Rate (%)'},
                     height=600)
    fig.update_layout(showlegend=True)
    fig.show()

    # 2. Interactive Bar Chart: College-wise Average Package
    college_avg = df.groupby('College').agg({
        'Avg_Package_LPA': 'mean',
        'Placement_Rate': 'mean',
        'Total_Students': 'sum'
    }).reset_index().round(2)

    fig = px.bar(college_avg.sort_values('Avg_Package_LPA', ascending=False),
                 x='College',
                 y='Avg_Package_LPA',
                 color='Placement_Rate',
                 hover_data=['Total_Students'],
                 title='Interactive: Average Package by College',
                 labels={'Avg_Package_LPA': 'Average Package (LPA)',
                        'Placement_Rate': 'Avg Placement Rate (%)'},
                 color_continuous_scale='Viridis',
                 height=600)
    fig.update_xaxes(tickangle=45)
    fig.show()

    # 3. Interactive Sunburst: Hierarchical view (College -> Field -> Year)
    fig = px.sunburst(df,
                      path=['College', 'Field', 'Year'],
                      values='Total_Students',
                      color='Avg_Package_LPA',
                      title='Interactive Sunburst: College → Field → Year Distribution',
                      color_continuous_scale='RdYlGn',
                      height=700)
    fig.show()

    # 4. Interactive Line Chart: Year-wise Trends by College
    year_college_trends = df.groupby(['Year', 'College']).agg({
        'Avg_Package_LPA': 'mean',
        'Placement_Rate': 'mean'
    }).reset_index()

    fig = px.line(year_college_trends,
                  x='Year',
                  y='Avg_Package_LPA',
                  color='College',
                  markers=True,
                  title='Interactive: Year-wise Package Trends by College',
                  labels={'Avg_Package_LPA': 'Average Package (LPA)'},
                  height=600)
    fig.show()

    # 5. Interactive Heatmap: College vs Field
    top_colleges = df.groupby('College')['Total_Students'].sum().nlargest(8).index
    top_fields = df.groupby('Field')['Total_Students'].sum().nlargest(10).index

    heatmap_data = df[df['College'].isin(top_colleges) & df['Field'].isin(top_fields)].pivot_table(
        values='Avg_Package_LPA',
        index='College',
        columns='Field',
        aggfunc='mean'
    )

    fig = px.imshow(heatmap_data,
                    title='Interactive Heatmap: Average Package (Top Colleges vs Fields)',
                    labels=dict(x='Field', y='College', color='Avg Package (LPA)'),
                    color_continuous_scale='RdYlGn',
                    aspect='auto',
                    height=600)
    fig.show()

    # 6. Interactive Box Plot: Package Distribution by Tier
    fig = px.box(df,
                 x='Tier',
                 y='Avg_Package_LPA',
                 color='Tier',
                 points='all',
                 title='Interactive: Package Distribution by Tier',
                 labels={'Avg_Package_LPA': 'Average Package (LPA)'},
                 category_orders={'Tier': ['Tier 1 (Premium)', 'Tier 2 (High)', 'Tier 3 (Average)']},
                 height=600)
    fig.show()

    print("✓ Interactive visualizations displayed successfully!")
else:
    print("⚠ Plotly not available. Install with: pip install plotly")

## 16. Summary

### Key Takeaways from the Analysis:

1. **Overall Placement Performance**
   - Total students analyzed across all colleges and years
   - Average placement rate and package trends identified
   - Year-over-year improvements or declines tracked

2. **Top Performers**
   - VJTI and SPIT consistently show high average packages
   - Computer Science and IT fields dominate in both placement rate and packages
   - Tier 1 colleges (Premium) offer packages ≥ ₹12 LPA

3. **Field-wise Insights**
   - Computer Science, Information Technology lead in placement opportunities
   - Traditional fields like Mechanical and Civil show lower average packages
   - Emerging fields (AI, Data Science) showing promising trends

4. **College-wise Insights**
   - Premier institutions maintain 70%+ placement rates
   - Package offerings vary significantly by college tier
   - Placement rates correlate moderately with average packages

5. **Temporal Trends**
   - Year 2023-2025 trends show evolution in placement landscape
   - Some colleges show consistent improvement
   - COVID-19 recovery visible in recent years

